In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum time loss

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from experimental_design import config
from sumo_grid_simulation.grid_simulation import Simulator

## Simulator

We start by initialising our simulation.

In [3]:
# trips_generator_period = 5 else simulation is v. slow
simulator = Simulator(trips_generator_period=5, end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_time_loss(X):
    # X = inputs - emukit doesnt pass named args, just an NxM ndarray, 
    # N is the number of points to evaluate, M is the number of parameters per each point
    
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')
    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = int(edgeLength),
            numberOfLanes = int(numLanes),
            accel         = accel
        )
        # average time loss / average route length
        result.append(s['timeLoss']/s['routeLength'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

In [5]:
def user_function_emissions(X):
    # X = inputs - emukit doesnt pass named args, just an NxM ndarray, 
    # N is the number of points to evaluate, M is the number of parameters per each point
    
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function emissions called with {X.shape[0]} inputs to simulate')
    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = int(edgeLength),
            numberOfLanes = int(numLanes),
            accel         = accel
        )
        # average time loss / average route length
        result.append(s['CO2']/s['routeLength'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [6]:
parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 25 datapoints from the parameter space:

In [7]:
init_X, init_Y = pickle.load(open('experimental_design/init_points/25_init_points.pkl', 'rb'))

'''
design = RandomDesign(parameter_space)  # initialize with random points
num_init_points = 5
init_X = design.get_samples(num_init_points)
init_Y = user_function_time_loss(init_X)
with open(f'experimental_design/init_points/{num_init_points}_init_points.pkl', "wb") as f:
      pickle.dump((init_X, init_Y), f)

'''

'\ndesign = RandomDesign(parameter_space)  # initialize with random points\nnum_init_points = 5\ninit_X = design.get_samples(num_init_points)\ninit_Y = user_function_time_loss(init_X)\nwith open(f\'experimental_design/init_points/{num_init_points}_init_points.pkl\', "wb") as f:\n      pickle.dump((init_X, init_Y), f)\n\n'

To then fit a GP to these points

In [8]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

Optimization restart 1/5, f = -67.56624250620654
Optimization restart 2/5, f = -58.26021108544671
Optimization restart 3/5, f = -58.26021108586314
Optimization restart 4/5, f = -58.26021108504591
Optimization restart 5/5, f = -88.03728038049297


### Experimentally designed emulator

We also load the emulator obtained during experimental design:

In [9]:
emukit_model_exp_design = pickle.load(open('experimental_design/models/25_init_50_loop_model_timeLoss.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [10]:
n_iter_bo = 10

### Acquisition functions

In [11]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [12]:
acquisition_exp_design = ExpectedImprovement(emukit_model_exp_design)

### Optimisation loop

In [13]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -88.03728038053276
Optimization restart 2/5, f = -88.03728037860688
Optimization restart 3/5, f = -88.03728038057528
Optimization restart 4/5, f = -58.26021108513707
Optimization restart 5/5, f = -58.260212700207745
[[ 20.    8.    5.  150.    1.    1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -91.3553107187527
Optimization restart 2/5, f = -60.8288483461555
Optimization restart 3/5, f = -91.35531071845892
Optimization restart 4/5, f = -60.828845315254945
Optimization restart 5/5, f = -91.35531071994589
[[ 20.           8.           5.         150.           1.
    1.68405458]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -94.84325021731051
Optimization restart 2/5, f = -94.84325021952378
Optimization restart 3/5, f = -94.84325021881763
Optimization restart 4/5, f = -

In [14]:
bo_exp_design = BayesianOptimizationLoop(parameter_space, emukit_model_exp_design, acquisition=acquisition_exp_design)
bo_exp_design.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -216.6480048232522
Optimization restart 2/5, f = -216.64800482522958
Optimization restart 3/5, f = -216.6480048245077
Optimization restart 4/5, f = -144.05609660383595
Optimization restart 5/5, f = -216.6480048227271
[[ 20.           8.          41.66449567 109.34905448   1.
    1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -220.2222955005013
Optimization restart 2/5, f = -146.43190831589425
Optimization restart 3/5, f = -146.4319083158942
Optimization restart 4/5, f = -146.43190833858444
Optimization restart 5/5, f = -220.22230189915567
[[ 20.           8.          43.52562272 112.78797953   1.
    1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -223.81394139657328
Optimization restart 2/5, f = -223.81394333033734
Optimization restart 3/5, f = -223.81

## Results

### Results on the random initialised emulator

In [15]:
results_random_init = bo_random_init.get_results()

In [16]:
results_random_init.minimum_location

array([ 14.96899422,  18.83841939,   9.18279573, 135.87276334,
         1.        ,   2.29420573])

In [17]:
results_random_init.minimum_value

0.010573522048808326

### Results on the experimentally designed emulator

In [18]:
results_exp_design = bo_exp_design.get_results()

In [19]:
results_exp_design.minimum_location

array([ 20.        ,   8.        ,  43.52562272, 112.78797953,
         1.        ,   1.5       ])

In [20]:
results_exp_design.minimum_value

0.010433652234397984